In [ ]:
%matplotlib inline

I was having a lot of trouble getting a numerically stable simulation based upon the QOBS SST profile. Even with a time step as small as 15 seconds, these simulations ulimately gave some sort of instability. The problem was worsened when the equator-to-pole SST gradient was larger. I suspect this is due to 

1. initializing the model very far from radiative convective equilibrium, and
2. the lack of convection parametrization leads to grid-scale precipitation.

To remove the second issue, I decided to test the dry dynamics of SAM using the standard Held Suarez benchmark [1]. 
In this notebook, I plot the results of a forcing the dry-dynamics of SAM with the Held-Suarez benchmark forcing. The goal of this forcing is to show the models ability to generate typical jet-like structures when forced towards a baroclincally unstable temperature distribution. I ran the SAM model with a resolution of $\Delta x = 160$ km and a time step of 120 seconds. Despite the small time step, this code ran relatively quickly because SAMs advection routines are cheap, and could generate 100 days of output in a couple of hours on a single processor.

[1]: Held, I. M. & Suarez, M. J. A Proposal for the Intercomparison of the Dynamical Cores of Atmospheric General Circulation Models. Bull. Am. Meteorol. Soc. 75, 1825–1830 (1994).

In [ ]:
# import glob, os
# caseid = "HeldSuarez_dt120"
# paths = glob.glob(f"../OUT_3D/{caseid}*.bin3D")
# path = paths[-1][3:]
# path = os.path.splitext(path)[0]

# ncpath = "../" + path + ".nc"
# !docker run -v  $(pwd)/../:/tmp -w /tmp nbren12/sam bin3D2nc {path}.bin3D

# ds = xr.open_dataset(ncpath)

In [ ]:
caseid = "HeldSuarez_dt120"
out_3d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_3D"

In [ ]:
ds = xr.open_mfdataset(f"{out_3d_path}/{caseid}_*.nc", autoclose=True)

# Climate

In [ ]:
mean = ds.sel(time=slice(70, None))\
         .mean(['time', 'x'])

In [ ]:
p = mean.p
tm = mean.TABS

tht  = tm * (1000/p)**(2/7)

levels = np.r_[270:350:5]

im = plt.contourf(tht.y, p, tht, levels=levels, extend='both')
plt.colorbar()
plt.clabel(im, colors='black', inline=False, fmt="%.0f")
plt.ylim([1000, 10])
plt.title("Potential Temperature")
plt.xlabel("y")
plt.ylabel("p")
# tht.plot.contourf(levels=levels)

In [ ]:
plt.contourf(tm.y, p, tm)
plt.ylim([1000,10])
plt.colorbar()
plt.title("Absolute Temperature")
plt.xlabel("y")
plt.ylabel("p");

In [ ]:
im = plt.contourf(mean.y, mean.p, mean.U,
                  levels=np.r_[-40:45:5], cmap='RdBu_r',
                  extend='both')


plt.contour(mean.y, mean.p, mean.U, levels=[-10, -8,-6,-4,-2,0], colors='k')
plt.ylim([1000,10])
plt.colorbar(im)
plt.title("Zonal Velocity")
plt.xlabel("y")
plt.ylabel("p");

# Spin-up

Now I plot the temporal evolution of some different fields at a lower atmosphere height as the simulation spins up

Here is the zonal velocity

In [ ]:
ds.U[::20, 5].plot(col='time', col_wrap=4)

and the meridional velocity

In [ ]:
ds.V[::20, 5].plot(col='time', col_wrap=4)

and the absolute temperature

In [ ]:
ds.TABS[::20, 5].plot(col='time', col_wrap=4)